# Aprendizado de Máquina: Processo de Linguagem Natural 

## Autor
**Nome**: Pedro Lucas Cassiano Martins<br>
**Matrícula**: 190036567<br>
**Github**: [PedroLucasCM](github.com/PedroLucasCM)

## Resumo
<p>O aprendizado de Máquina vem crescendo muito no mercado desde a última decada e é uma área da computação/informática muito estudada também, por isso estamos estudando essa área no curso de Engenharia de Software na Faculdade do Gama da Universidade de Brasília.</p>
<p>Esse artigo foi criado no intuito de continuar este estudo ao usar Natural Language Process (NLP), que é uma área de deep learning que vem crescendo e melhorando drasticamente nos últimos anos, e focando na biblioteca Transformers do HuggingFace ao invés da biblioteca do *fast.ai*.</p>
A maior utilidade do NLP podemos dizer que é a classificação, que é classificação automática de documentos em categorias. 

**O estudo é majoritariamente baseado no curso "_fast.ai_" de Jeremy Howard, mencionado na blibliografia desse artigo.**

## Objetivo
<p>O objetivo deste artigo é o de estudar Natural Language Process (NLP) além de não se acostumar em usar apenas uma biblioteca para tudo.</p>


## Atualização e Imports

In [1]:
#|export
import os
import warnings
from pathlib import Path

# Código para ignorar avisos de usuário
warnings.filterwarnings("ignore", category=UserWarning)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    path = Path('../input/proconsumidores')
    ! pip install -Uqq datasets
    ! pip install -Uqq evaluate

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from collections import Counter
from transformers import TrainingArguments,Trainer
import numpy as np, matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
import evaluate

warnings.filterwarnings("ignore", category=UserWarning)

## NLP

Modelos de classificação também podem ser usados para resolver problemas que não são, a princípio, obviamente apropriados. 

Aqui usaremos uma planilha com nomes de inscritos em um curso da ENDC (Escola Nacional de Defesa do Consumidor) onde aparecem os estados, procons e a situação de cadastro das pessoas inscritas. As situações variam entre OK, JÁ FEZ, CADASTRO INCOMPLETO e SEM CADASTRO. Temos também uma coluna score onde quem tem cadastro tem score 1 e quem não tem cadastro tem score 0.

Documentos em conjuntos de dados NLP geralmente estão em uma das duas formas principais:

* Larger Documents: Um arquivo de texto por documento, geralmente organizado em uma pasta por categoria
* Smaller Documents: um documento (ou par de documentos, opcionalmente com metadados) por linha em um arquivo CSV.

Vamos olhar para os nossos dados e ver o que temos. No Jupyter, você pode usar qualquer comando bash/shell iniciando uma linha com um ***!*** e usar ***{ }*** para incluir variáveis *​​python*, assim:

In [3]:
!ls {path}

proconsumidor.csv


Essa competição utiliza arquvios csv, e por isso importamos a biblioteca 'panda' mais cedo.

Colocaremos então um path para o arquivo utilizando a função do panda *read_csv*. Isso cria um *DataFrame* que é uma tabela com colunas nomeadas, um pouco parecido com tabela de base de dados.

In [4]:
df = pd.read_csv(path/'proconsumidor.csv')

Para ver as primeiras e ultimas linhas do DataFrame é só digitar o nome que colocamos para o path.

In [5]:
df.columns = df.columns.str.strip()
df.columns

Index(['NOME', 'ORGAO', 'ESTADO', 'SITUACAO', 'SCORE'], dtype='object')

In [6]:
df.rename(columns={'NOME': 'nome', 'ORGAO': 'orgao', 'ESTADO': 'estado', 'SITUACAO': 'situacao', 'SCORE': 'score'}, inplace=True)

In [7]:
df['score'] = df['score'].astype(float)
df

,nome,orgao,estado,situacao,score
0,ANA CECILIA NEGREIROS DUNCAN MACHADO,PROCON ESTADUAL DO MATO GROSSO DO SUL,MS,OK,1.0
1,Guilherme Augusto de Castro Machado,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0
2,Naiara Aparecida de Souza,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0
3,Naiane Souza Mendonça,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0
4,LUCAS GONÇALVES DINIZ,PROCON MUNICIPAL DE CAÇAPAVA,SP,OK,1.0
...,...,...,...,...,...
169,Isabella de Oliveira David,PROCON ESTADUAL DE PERNAMBUCO,PE,OK,1.0
170,Ewerton de Melo Farias,PROCON ESTADUAL DE PERNAMBUCO,PE,OK,1.0
171,DIESSICA SAYWRI MEIRA GOMES,PROCON ESTADUAL DO MATO GROSSO DO SUL,MS,OK,1.0
172,GABRIELA TIEMI SHINGO,PROCON MUNICIPAL DE CASCAVEL,PR,OK,1.0


Existem algumas linhas sem valores, por isso deletaremos elas para não dar erro.

Uma das features mais úteis do *DataFrame* é o método "describe()":

In [8]:
df.describe(include='object')

,nome,orgao,estado,situacao
count,174,171,171,174
unique,174,56,15,4
top,ANA CECILIA NEGREIROS DUNCAN MACHADO,PROCON MUNICIPAL DE PORTO ALEGRE,RS,OK
freq,1,17,39,131


Podemos ver que nas 174 linhas, existem 174 nomes únicos, 58 órgãos, 15 estados únicos e, o principal, 4 situações. O mais comum aqui seriam as situações tendo 131 aparições de OKs.

Anteriormente, sugeri que poderíamos representar a entrada para o modelo como algo como **"TEXT1: abatement; TEXT2: eliminating process"**. Precisamos adicionar o contexto a isso também. No *Pandas*, usamos apenas + para concatenar, assim:

In [9]:
df['input'] = 'NOME: ' + df.nome + '; ÓRGÃO: ' + df.orgao + '; ESTADO: ' + df.estado  + '; SITUAÇÃO: ' + df.situacao

Agora, para pegar as primeiras linhas podemos utilizar o método ***head()***:

In [10]:
df

,nome,orgao,estado,situacao,score,input
0,ANA CECILIA NEGREIROS DUNCAN MACHADO,PROCON ESTADUAL DO MATO GROSSO DO SUL,MS,OK,1.0,NOME: ANA CECILIA NEGREIROS DUNCAN MACHADO; ÓR...
1,Guilherme Augusto de Castro Machado,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0,NOME: Guilherme Augusto de Castro Machado; ÓRG...
2,Naiara Aparecida de Souza,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0,NOME: Naiara Aparecida de Souza; ÓRGÃO: PROCON...
3,Naiane Souza Mendonça,PROCON MUNICIPAL DE IBIRITÉ,MG,OK,1.0,NOME: Naiane Souza Mendonça; ÓRGÃO: PROCON MUN...
4,LUCAS GONÇALVES DINIZ,PROCON MUNICIPAL DE CAÇAPAVA,SP,OK,1.0,NOME: LUCAS GONÇALVES DINIZ; ÓRGÃO: PROCON MUN...
...,...,...,...,...,...,...
169,Isabella de Oliveira David,PROCON ESTADUAL DE PERNAMBUCO,PE,OK,1.0,NOME: Isabella de Oliveira David; ÓRGÃO: PROCO...
170,Ewerton de Melo Farias,PROCON ESTADUAL DE PERNAMBUCO,PE,OK,1.0,NOME: Ewerton de Melo Farias; ÓRGÃO: PROCON ES...
171,DIESSICA SAYWRI MEIRA GOMES,PROCON ESTADUAL DO MATO GROSSO DO SUL,MS,OK,1.0,NOME: DIESSICA SAYWRI MEIRA GOMES; ÓRGÃO: PROC...
172,GABRIELA TIEMI SHINGO,PROCON MUNICIPAL DE CASCAVEL,PR,OK,1.0,NOME: GABRIELA TIEMI SHINGO; ÓRGÃO: PROCON MUN...


In [11]:
df.input

0      NOME: ANA CECILIA NEGREIROS DUNCAN MACHADO; ÓR...
1      NOME: Guilherme Augusto de Castro Machado; ÓRG...
2      NOME: Naiara Aparecida de Souza; ÓRGÃO: PROCON...
3      NOME: Naiane Souza Mendonça; ÓRGÃO: PROCON MUN...
4      NOME: LUCAS GONÇALVES DINIZ; ÓRGÃO: PROCON MUN...
                             ...                        
169    NOME: Isabella de Oliveira David; ÓRGÃO: PROCO...
170    NOME: Ewerton de Melo Farias; ÓRGÃO: PROCON ES...
171    NOME: DIESSICA SAYWRI MEIRA GOMES; ÓRGÃO: PROC...
172    NOME: GABRIELA TIEMI SHINGO; ÓRGÃO: PROCON MUN...
173    NOME: ALEXANDRE APARECIDO DA SILVA; ÓRGÃO: PRO...
Name: input, Length: 174, dtype: object

In [12]:
df.iloc[0,4]

1.0

Usaremos a seguinte função para buscar uma situação de sem cadastro onde o SCORE vai ser 0.

In [13]:
df[df['score'] == 0]["situacao"].values[1]

'OK'

A df.columns mostra as colunas, poderemos ver que a coluna SITUAÇÃO tem um espaço a mais no fim. Usaremos a df.columns.str.strip() para tirar os espaços adicionais em todas as colunas

In [14]:
df = df.dropna()

## Tokenização

A biblioteca Transformers do HuggingFace uitiliza um objeto de *Dataset* para guardar um Dataset. Podemos criar um assim:

In [15]:
dsf = Dataset.from_pandas(df)

In [16]:
dsf

Dataset({
    features: ['nome', 'orgao', 'estado', 'situacao', 'score', 'input', '__index_level_0__'],
    num_rows: 171
})

Mas não podemos passar os textos diretamente para um modelo. Um modelo de aprendizado profundo espera números como entradas, não frases em inglês! Portanto, precisamos fazer duas coisas:

- *Tokenização*: Dividir cada texto em palavras (ou na verdade, como veremos, em *tokens*)
- *Numerização*: Converta cada palavra (ou token) em um número.

Os detalhes sobre como isso é feito dependem do modelo específico que usamos. Então, primeiro precisamos escolher um modelo. Existem milhares de modelos disponíveis, mas um ponto de partida razoável para quase qualquer problema de PNL é usar isso (substitua "pequeno" por "grande" para obter um modelo mais lento, porém mais preciso, assim que terminar de explorar):

In [17]:
model_nms = 'microsoft/deberta-v3-small'

`AutoTokenizer` vai criar um tokenizador apropriado para o modelo dado, no caso o model_nm acima:

In [18]:
tokz = AutoTokenizer.from_pretrained(model_nms)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Palavras incomuns serão divididas em pedaços. O início de uma nova palavra é representado por `▁`:

In [19]:
tokz.tokenize('NOME: ANA CECILIA NEGREIROS DUNCAN MACHADO; ÓRGÂO: PROCON ESTADUAL DO MATO GROSSO DO SUL ; ESTADO: MS')

['▁NO',
 'ME',
 ':',
 '▁ANA',
 '▁CEC',
 'ILIA',
 '▁NE',
 'GRE',
 'I',
 'ROS',
 '▁DUN',
 'CAN',
 '▁MACH',
 'ADO',
 ';',
 '▁Ó',
 'RG',
 'Â',
 'O',
 ':',
 '▁PRO',
 'CON',
 '▁EST',
 'AD',
 'UAL',
 '▁DO',
 '▁M',
 'ATO',
 '▁GROSS',
 'O',
 '▁DO',
 '▁S',
 'UL',
 '▁;',
 '▁EST',
 'ADO',
 ':',
 '▁MS']

In [20]:
def tok_func(x): return tokz(x['orgao'])
tok_ds = dsf.map(tok_func, batched=True)

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Para executar isso rapidamente em paralelo em cada linha do nosso conjunto de dados, use `map`:

Isso adiciona um novo item ao dataset chamado de `input_ids`. Por exemplo, aqui está o input e IDs da primeira linha do nosso dado:

In [21]:
dsf

Dataset({
    features: ['nome', 'orgao', 'estado', 'situacao', 'score', 'input', '__index_level_0__'],
    num_rows: 171
})

In [22]:
tokz

DebertaV2TokenizerFast(name_or_path='microsoft/deberta-v3-small', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [23]:
row = tok_ds[0]
row['orgao'], *row['input_ids']

('PROCON ESTADUAL DO MATO GROSSO DO SUL',
 1,
 12535,
 18638,
 14266,
 13660,
 50022,
 6745,
 749,
 58008,
 90716,
 1702,
 6745,
 662,
 18343,
 2)

Então, quais são esses IDs e de onde eles vêm? O segredo é que há uma lista chamada `vocab` no tokenizer que contém um inteiro único para cada token string possível. Podemos procurá-los assim, por exemplo, para encontrar o token para a palavra "of":

In [24]:
tokz.vocab['▁PRO']

12535

Olhando acima para nossos IDs de entrada, vemos que `265` aparece como esperado.

Finalmente, precisamos preparar nossos rótulos. Transformers sempre assume que seus rótulos têm o nome de coluna `labels`, mas em nosso conjunto de dados é atualmente `score`. Portanto, precisamos renomeá-lo:

In [25]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

Agora que preparamos nossos tokens e rótulos, precisamos criar nosso conjunto de validação.

## Sets testes e de validação

### Validação

Para evitar *overfitting*, que é quando uma função fica ajustada demais ao conjunto de dados em que foi treinada e acaba desviando do formato real da função que estava sendo aproximada, é importante separar um conjunto de dados para não incluir no treinamento, pois não queremos que o modelo o veja.

Isso é chamado de *conjunto de validação*, e a biblioteca Transformers disponibiliza o `train_test_split()` para sua criação:

In [26]:
dds = tok_ds.train_test_split(0.25, seed=42) # 25% para validação e 75% para treino
dds

DatasetDict({
    train: Dataset({
        features: ['nome', 'orgao', 'estado', 'situacao', 'labels', 'input', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 128
    })
    test: Dataset({
        features: ['nome', 'orgao', 'estado', 'situacao', 'labels', 'input', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 43
    })
})

Aqui o conjunto de validação é chamado `test`.

### Teste

O *conjunto de teste* também é isento do treino, e só é usado após concluído todo o processo de treinamento.

A medida em que você tenta coisas diferentes para ver o impacto nas métricas do teste de validação, é possível que encontre algumas coisas que acidentalmente melhorem os resultados naquela situação, mas que no mundo real não são exatamente melhores. Por isso o conjunto de teste serve para mostrar se ocorreu *overfitting* com o conjunto de validação.

In [27]:
eval_df = df
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Chamamos de `eval` para não confundir com o outro conjunto criado acima.

## Metricas

Quando estamos treinando um modelo, haverá uma ou mais métricas que estamos interessados em maximizar ou minimizar. Estas são as medições que devem, esperançosamente, representar o quão bem nosso modelo funcionará para nós.

Na vida real, fora do Kaggle, as coisas não são fáceis... Como observa Dra. Rachel Thomas, em [O problema com as métricas é um grande problema para a IA](https://www.fast.ai/posts/2019-09-24-metrics.html):

> No fundo, o que as abordagens de IA mais atuais fazem é otimizar as métricas. A prática de otimizar métricas não é nova nem exclusiva da IA, mas a IA pode ser particularmente eficiente (e até eficiente demais!) ao fazê-lo. É importante entender isso, porque quaisquer riscos de otimização de métricas são aumentados pela IA. Embora as métricas possam ser úteis em seu devido lugar, há danos quando são aplicadas sem pensar. Algumas das instâncias mais assustadoras de algoritmos executados de forma descontrolada resultam de métricas excessivamente enfatizadas. Temos que entender essa dinâmica para entender os riscos urgentes que enfrentamos devido ao uso indevido da IA.

Poderemos usar uma métrica como a seguir usado em outro competição que utiliza também de 0 e 1 como o `score` nosso.

$$F1=\frac{2*Precision*Recall}{Precision+Recall}$$

onde:

$$Precision=\frac{TP}{TP+FP}$$

$$Recall=\frac{TP}{TP+FN}$$
e:
```
True Positive [TP] = sua previsão é 1, e a correta é 1 - você preveu positivo, portanto é verdadeiro.
False Positive [FP] = sua previsão é 1, e a correta é 0 - você preveu positivo, portanto é falso.
False Negative [FN] = sua previsão é 0, e a correta é 1 - você preveu negativo, portanto é falso.
```


já que o score tem uma variação de 1 ou 0 apenas.

In [28]:
def compute_metrics(eval_preds):
    metric = evaluate.load('f1')
    logits, labels = eval_preds
    predictions = np.clip(logits, 0, 1)
    return metric.compute(predictions=predictions, references=labels)

## Treinamento

Primeiro definimos o *batch size*, o número de épocas, e a taxa de aprendizado:

In [29]:
bs = 64
epochs = 4
lr = 8e-5

A seguir criamos os argumentos de treinamento, contendo as variáveis definidas anteriormente e algumas configurações utilizando o `TrainingArguments`:

In [30]:
args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True, 
    evaluation_strategy="epoch", 
    per_device_train_batch_size=bs, 
    per_device_eval_batch_size=bs*2, 
    num_train_epochs=epochs, 
    weight_decay=0.01, 
    report_to='none'
    )

Podemos agora criar o modelo e o `Trainer`, uma classe que combina dados e modelo juntos (como o Learner do fastai):

In [52]:

model = AutoModelForSequenceClassification.from_pretrained(model_nms, num_labels=1)
trainer = Trainer(
    model, 
    args, 
    train_dataset=dds['train'], 
    eval_dataset=dds['test'],
    tokenizer=tokz, 
    compute_metrics=compute_metrics
    )

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Ignorando as warnings, agora treinaremos o modelo:

In [32]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.640262,0.000000
2,No log,0.417636,0.000000
3,No log,0.183504,0.000000
4,No log,0.171062,0.000000


TrainOutput(global_step=8, training_loss=0.5055265426635742, metrics={'train_runtime': 4.9504, 'train_samples_per_second': 103.425, 'train_steps_per_second': 1.616, 'total_flos': 2384455587840.0, 'train_loss': 0.5055265426635742, 'epoch': 4.0})

In [33]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[0.89208984],
       [0.90283203],
       [0.90283203],
       [0.90283203],
       [0.90576172],
       [0.88916016],
       [0.90429688],
       [0.90429688],
       [0.90429688],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90625   ],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.90771484],
       [0.89697266],
       [0.88671875],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90576172],
       [0.90576172],
       [0.90771484],
       [0.908

Como nosso gabarito não possui valores maiores do que 1 e menores do que 0, utilizamos o `np.clip()` para ficarmos com valores entre 0 e 1.

In [34]:
preds = np.clip(preds, 0, 1)
preds

array([[0.89208984],
       [0.90283203],
       [0.90283203],
       [0.90283203],
       [0.90576172],
       [0.88916016],
       [0.90429688],
       [0.90429688],
       [0.90429688],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90917969],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90673828],
       [0.90625   ],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.89941406],
       [0.90771484],
       [0.89697266],
       [0.88671875],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90771484],
       [0.90576172],
       [0.90576172],
       [0.90771484],
       [0.908

Para submeter nossas previsões para o Kaggle, devemos gerar um arquivo .csv semelhante ao template disponibilizado, então vamos transformar os itens de `preds` em `int`:

In [35]:
preds = [int(x) for x in preds]

In [36]:
import datasets

submission = Dataset.from_dict({
    'nome': eval_ds['nome'],
    'orgao': eval_ds['orgao'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

10190

Agora podemos ver o arquivo gerado:

In [37]:
sub = pd.read_csv('submission.csv')
sub

,nome,orgao,score
0,ANA CECILIA NEGREIROS DUNCAN MACHADO,PROCON ESTADUAL DO MATO GROSSO DO SUL,0
1,Guilherme Augusto de Castro Machado,PROCON MUNICIPAL DE IBIRITÉ,0
2,Naiara Aparecida de Souza,PROCON MUNICIPAL DE IBIRITÉ,0
3,Naiane Souza Mendonça,PROCON MUNICIPAL DE IBIRITÉ,0
4,LUCAS GONÇALVES DINIZ,PROCON MUNICIPAL DE CAÇAPAVA,0
...,...,...,...
166,Isabella de Oliveira David,PROCON ESTADUAL DE PERNAMBUCO,0
167,Ewerton de Melo Farias,PROCON ESTADUAL DE PERNAMBUCO,0
168,DIESSICA SAYWRI MEIRA GOMES,PROCON ESTADUAL DO MATO GROSSO DO SUL,0
169,GABRIELA TIEMI SHINGO,PROCON MUNICIPAL DE CASCAVEL,0


# Conclusão

Essa lição demandou um pouco mais de domínio no conteúdo e muito tempo. Ocorreram vários erros durante o processo o que resultou em frustração e um artigo corrido.

Porém aprendi sobre como fazer análise com documentos, apesar de ter sido bem porco, porém isso foi falta de atenção e dedicação.

## Inferência

Aqui está o link para o meu [HuggingFace app](https://huggingface.co/spaces/Herises/FastaionCampus).

In [57]:
import torch
from transformers import DebertaV2ForSequenceClassification

torch.save(model.state_dict(), "model.pk1")

## Bibliografia

Howard, Jeremy. **Practical Deep Learning for Coders, 2022**. Disponível em: https://course.fast.ai/. Acesso em: 02 de abr. de 2023.

Howard, Jeremy. **Lesson 4: Natural Language Process (NLP), 2022**. Disponível em: https://course.fast.ai/Lessons/lesson4.html. Acesso em: 15 de mai. de 2023.

Howard, Jeremy. **Lesson 4: Practical Deep Learning for Coders 2022**, 2022. Disponível em: https://www.youtube.com/watch?v=toUgBQv1BT8s. Acesso em: 15 de mai. de 2023.

Howard, Jeremy. **Getting started with NLP for absolute beginners**, 2022. Dísponivel em: https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners. Acesso em 15 de maio de 2023.

Dra. Thomas, Rachel. **O problema com as métricas é um grande problema para a IA**, 24 de Set. de 2019. Disponivel em: https://www.fast.ai/posts/2019-09-24-metrics.html. Acessado em 17 de maio de 2023.